In [ ]:
from pyspark.sql import SparkSession as Session
from pyspark import SparkConf as Conf
from pyspark import SparkContext as Context

In [ ]:
import os
os.environ['SPARK_LOCAL_IP']='192.168.1.2'
os.environ['HADOOP_HOME']='/home/geno1664/Developments/Github_Samples/RDS-ENV/hadoop'
os.environ['LD_LIBRARY_PATH']='$LD_LIBRARY_PATH:$HADOOP_HOME/lib/native'
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'
os.environ['PYSPARK_PYTHON']='python3'
os.environ['PYARROW_IGNORE_TIMEZONE']='1'

In [ ]:
configuration = Conf().setAppName('RDS_1').setMaster('spark://GenoMachine:7077')
configuration.set('spark.executor.memory','10G').set('spark.driver.memory', '2G').set('spark.cores.max', '8')

In [ ]:
context = Context(conf=configuration)

In [ ]:
session = Session(context)

In [ ]:
from Functions.IO import CSV_File
mainCSV = CSV_File(session, r'/home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No1/IO/County_Classificiations.csv')

In [ ]:
mainCSV.Schema()

In [ ]:
from databricks import koalas as ks
mainDF = mainCSV.GetSparkDF().select('State','County','Population_loss_2015_update','Type_2015_Farming_NO','Type_2015_Manufacturing_NO','Type_2015_Recreation_NO','Type_2015_Mining_NO')

In [ ]:
mainDF = mainDF.repartition(50, 'State')

In [ ]:
mainDF = mainDF.withColumnRenamed('Type_2015_Farming_NO', 'Farming')
mainDF = mainDF.withColumnRenamed('Type_2015_Manufacturing_NO', 'Manufacturing')
mainDF = mainDF.withColumnRenamed('Type_2015_Recreation_NO', 'Recreation')
mainDF = mainDF.withColumnRenamed('Type_2015_Mining_NO', 'Mining')
mainDF = mainDF.withColumnRenamed('Population_loss_2015_update', 'Pop_Loss')

In [ ]:
mainDF = mainDF.dropna(how='any', subset=['State', 'County', 'Pop_Loss'])
mainDF = mainDF.dropna(how='all', subset=['Farming', 'Manufacturing', 'Recreation', 'Mining'])

In [ ]:
mainDF.show(n=25)

In [ ]:
farmingDF = mainDF.select('Pop_Loss', 'Farming').groupBy('Farming').mean()
manufacturingDF = mainDF.select('Pop_Loss', 'Manufacturing').groupBy('Manufacturing').mean()
recreationDF = mainDF.select('Pop_Loss', 'Recreation').groupBy('Recreation').mean()
miningDF = mainDF.select('Pop_Loss', 'Mining').groupBy('Mining').mean()

In [ ]:
Pop_Losses = {}

In [ ]:
Pop_Losses['Farming'] = farmingDF.first()['avg(Pop_Loss)']
Pop_Losses['Manufacturing'] = manufacturingDF.first()['avg(Pop_Loss)']
Pop_Losses['Recreation'] = recreationDF.first()['avg(Pop_Loss)']
Pop_Losses['mining'] = miningDF.first()['avg(Pop_Loss)']

In [ ]:
print(Pop_Losses)